In [ ]:
!python cerebrum/run_agent.py \
--mode local \
--agent_path cerebrum/example/agents/test_agent \
--task "What is the capital of United States?"

In [ ]:
!python cerebrum/run_agent.py \
--mode local \
--agent_path cerebrum/example/agents/math_agent \
--task "Use Wikipedia to look up the closed-form formula for the nth Catalan number, then compute the 15th Catalan number step by step."

In [ ]:
import requests
import json

def test_post(base_url, endpoint, payload):
    """
    Test the POST helper by sending a JSON payload to the given endpoint.
    Prints request details and response.
    """
    url = base_url.rstrip("/") + endpoint
    print(f"Request URL: {url}")
    # print("Request payload:", json.dumps(payload, indent=2, ensure_ascii=False))

    try:
        resp = requests.post(url, json=payload)
    except Exception as e:
        print(f"Error during HTTP request: {e}")
        return

    print("Response status code:", resp.status_code)
    print("Response text:", resp.text)

    try:
        data = resp.json()
        print("Parsed JSON response:", json.dumps(data, indent=2, ensure_ascii=False))
    except ValueError as e:
        print("Failed to parse JSON:", e)

if __name__ == "__main__":
    # Configure base URL and endpoint for your AIOS kernel
    base_url = "http://localhost:8000"
    endpoint = "/query"

    # Construct payload based on observed llm_query log
    payload = {
        "query_type": "llm",
        "agent_name": "math_agent",
        "query_data": {
            "llms": None,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an expert who is good at solving mathematical problems. You can help users understand and solve various mathematical problems by searching relevant research papers and providing step-by-step solutions."
                },
                {
                    "role": "user",
                    "content": "Solve the system of equations: 2x + 3y = 6 and 4x - y = 5, and provide a step-by-step solution."
                },
                {
                    "role": "user",
                    "content": "[Thinking]: The workflow generated for the problem is [{\"action_type\": \"tool_use\", \"action\": \"Search  closed-form formula for the nth Catalan number\", \"tool_use\": [\"wikipedia\"]}, {\"action_type\": \"chat\", \"action\": \"Analyze the mathematical problem and provide solution steps\", \"tool_use\": []}, {\"action_type\": \"chat\", \"action\": \"Calculate and verify the final answer\", \"tool_use\": []}]. Follow the workflow to solve the problem step by step."
                },
                {
                    "role": "user",
                    "content": "At step 1, you need to: Search for relevant mathematical concepts and formulas."
                }
            ],
            "tools": [
                {
                    "type": "function",
                    "function": {
                        "name": "wikipedia",
                        "description": "Search information on the Wikipedia",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "query": {
                                    "type": "string",
                                    "description": "Query used for searching information on the Wikipedia"
                                }
                            },
                            "required": ["query"]
                        }
                    }
                }
            ],
            "action_type": "tool_use",
            "message_return_type": "text",
            "response_format": None
        }
    }

    test_post(base_url, endpoint, payload)

In [ ]:
from pathlib import Path
from cerebrum.manager.tool import ToolManager

manager = ToolManager(base_url='https://app.aios.foundation')

# 举例：加载一个 core 工具
tool_cls, cfg = manager.load_tool(local=True, name="google_search")
print(tool_cls)  # <class 'cerebrum.tool.core.bing_search.BingSearch'>
print(cfg)       # None

# 如果你想真正“拿来用”，需要先实例化：
tool = tool_cls()          # 或者 tool_cls(**cfg) 如果 cfg 不是 None
result = tool.run(params={ "query": "你好，世界" })